In [1]:
%matplotlib notebook
import dataAcquisition as daq
import matplotlib.pyplot as plt
from time import perf_counter,time,sleep
from numpy import median,std

In [2]:
port = '/dev/ttyUSB0'
res = [9.88,10.04,9.83,10,9.97,10]
dur = 60
step = 1

hal9001 = daq.DAQ(port,res,dur,step)

a = 9.57e-04
b = 3.15e-04
c = -1.17e-05
d = 6.23e-07

readings nominal


In [7]:
for i in hal9001:
    volt = i.readVoltage()
    res = i.calcResistance(volt)
    temp = i.calcTemperature(res,a,b,c,d)
    print(i,temp - 273.15)

<dataAcquisition.Channel object at 0x7fccc01b76d8> 65.03011953694727
<dataAcquisition.Channel object at 0x7fccc01b7748> 64.98358670297125
<dataAcquisition.Channel object at 0x7fccc01b77b8> 65.54371029260369
<dataAcquisition.Channel object at 0x7fccc01b7828> 65.02315919137266
<dataAcquisition.Channel object at 0x7fccc01b78d0> 64.92240896585088
<dataAcquisition.Channel object at 0x7fccc01b7940> 62.245486178328974


In [ ]:
plt.ion()

start_time = time()
timepoints = []
view_time = 5
duration = dur
pause = step

timeDuraction = {
    'loop':[],
    'reads':[],
    'plot':[],
    'save':[],
    'pause':[]
}

fig1 = plt.figure()
# http://matplotlib.org/users/text_props.html
fig1.suptitle('live updated data', fontsize='18', fontweight='bold')
plt.xlabel('Time [s]', fontsize='14', fontstyle='italic')
plt.ylabel('Temperature [deg. C]', fontsize='14', fontstyle='italic')
plt.axes().grid(True)

styles = ['rd','bo','ks','g^','y+','mp']
labels = ['T1','T2','T3','T4','T5','T6']

plt.xlim([0,view_time])

run = True;

counter = 0
while run:
    startLoop = perf_counter()
    timepoints.append(time()-start_time)
    current_time = timepoints[-1]
    
    j = 0
    startRead = perf_counter()
    for i in hal9001:
        temp = i.calcTemperature(i.calcResistance(i.readVoltage()),a,b,c,d)
        i.recoValue(counter,temp)
        # update the plotted data
        plt.plot(timepoints,i.data[i.data!=0],styles[j])
        j+=1
    endRead = perf_counter()
    timeDuraction['reads'].append(endRead-startRead)
    counter+=1
    
    startSave = perf_counter()
    hal9001.saveData()
    endSave = perf_counter()
    timeDuraction['save'].append(endSave-startSave)
        
    # slide the viewing frame along
    if current_time > view_time:
        plt.xlim([current_time-view_time,current_time])
    
    if timepoints[-1] > duration: run=False
        
    # update the plot
    startPlot = perf_counter()
    fig1.canvas.draw()
    endPlot = perf_counter()
    timeDuraction['plot'].append(endPlot-startPlot)
    
    startPause = perf_counter()
    sleep(pause)
    endPause = perf_counter()
    timeDuraction['pause'].append(endPause-startPause)
    
    endLoop = perf_counter()
    timeDuraction['loop'].append(endLoop-startLoop)
    
hal9001.cleanData()
startSave = perf_counter()
hal9001.saveData()
endSave = perf_counter()
timeDuraction['save'].append(endSave-startSave)
    
# # plot all of the data you collected
# fig2 = plt.figure()
# # http://matplotlib.org/users/text_props.html
# fig2.suptitle('Complete Data Trace', fontsize='18', fontweight='bold')
# plt.xlabel('Time [s]', fontsize='14', fontstyle='italic')
# plt.ylabel('Temperature [deg. K]', fontsize='14', fontstyle='italic')
# plt.axes().grid(True)

# hal9001.cleanData()
# for i in hal9001:
#     plt.plot(timepoints,i.data,styles[i],label=labels[i])
# plt.legend()
# fig2.show()

### Problem 1

In [ ]:
fig2, ((ax1),(ax2)) = plt.subplots(1,2,figsize=(20,10))
colors = ['r-D','b-o','k-s','g-^','y-+']
j = 0
for i in timeDuraction:
    currentLabel = i + ': %1.3f +/- %1.3f' %
    (median(timeDuraction[i]),std(timeDuraction[i]))
    
    ax1.plot(timepoints,timeDuraction[i],colors[j],label=currentLabel)
        
    ax2.semilogy(timepoints,timeDuraction[i],colors[j],label=i)
    j+=1
    
ax1.legend()
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Duration [s]')
ax1.set_title('linear')

ax2.legend()
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Duration [s]')
ax2.set_title('semilogy')

fig2.savefig('problem1.png')

In [ ]:
for i in timeDuraction:
    value = '%1.3f +/- %1.3f' % (median(timeDuraction[i]),std(timeDuraction[i]))
    print(i,value)

### Problem 2

In [ ]:
import numpy as np

diffV = np.arange(0,5,.005)

def temp(a,b,c,d,res):
    return (a+b*np.log(res)+c*np.log(res)**2+d*np.log(res)**3)**(-1)

def res(voltRead):
    return voltRead/(5-voltRead)*10

diffT = temp(a,b,c,d,res(diffV))

delT = []
for i in range(1,len(diffT)):
    delT.append(diffT[i-1]-diffT[i])

In [ ]:
delT100 = []
for i in delT:
    if(i > 0 and i <= 1):
        delT100.append(i)
T = np.linspace(0,100,len(delT100))

In [ ]:
fig3 = plt.figure(figsize=(20,10))
plt.plot(T,delT100,'o')
plt.xlabel('Temperature, T[deg.C]')
plt.ylabel('Resolution, delT [deg.C]')
fig3.savefig('problem2.png')

### Conclusions
#### Problem 1
I put the median and standard deviation in my legend to get a better idea of the time variance. My largest factor with the largest variability seems to be my loop currently, which makes sense, because I know I currently have some extra functions that allow me to plot in real-time. The '%matplotlib notebook' at the top of my workbook is an extra line I inserted, which puts my plots in javascript wrapper to be rendered in real-time along with me using plt.ion() to allow interaction. This unfortunately also slows down plotting (which also has a large std) which leads to why my loop has a larger std. I am addressing this by making my own GUI, but didn't finish it in time to compare for a time check against this wapper.